# 8760 Notebook

### Imports

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cftime
import xarray as xr
import hvplot.xarray
import holoviews as hv
import panel as pn
from typing import List, Union, Tuple
from climakitae.explore.vulnerability import cava_data
from climakitae.util.utils import add_dummy_time_to_wl
from climakitae.explore.threshold_tools import (
    get_block_maxima,
    get_return_value,
    get_ks_stat,
    _get_fitted_distr,
    _calculate_return,
    _conf_int,
    _get_distr_func,
)
from phase1 import *
random.seed(42)

### Global Variables

In [ ]:
# Putting locations into a DataFrame
locations = {
    "goleta": (34.486652, -119.894310),
    "iv": (34.427741, -119.855950),
    "sb_mountains": (34.518453, -119.799645),
    "valencia": (34.432468, -118.571098),
    "castaic": (34.492141, -118.605053),
    "lebec": (34.680102, -118.789246),
}

df = pd.DataFrame.from_dict(locations, orient='index', columns=['lat', 'lon']).reset_index()
df.rename(columns={'index': 'location'}, inplace=True)

In [ ]:
# Santa Barbara area
sb = df.iloc[:3][['lat', 'lon']]
sb_locs = ['goleta', 'iv', 'sb_mountains']

# Santa Clarita area
sc = df.iloc[3:6][['lat', 'lon']]
sc_locs = ['valencia', 'castaic', 'lebec']

In [ ]:
# Set params for 1-in-X calculations
one_in_x = np.array([10, 100])
event_duration = (1, 'day')
distr = 'gev'

### Grabbing appropriate datasets

In [ ]:
locs = [sb, sc]
datas = []
for loc in locs:
    data = cava_data(
        ## Set-up
        loc, # select a single location
        variable="Air Temperature at 2m",
        units="degF", # change units
        downscaling_method="Dynamical",  # WRF data 
        approach="Warming Level",  
        warming_level=2.0,
        wrf_bias_adjust=False, # return bias adjusted WRF models
    
        # These are passed in but the export method is 'raw', so no calculation is done
        # This is because `cava_data` doesn't currently provide lower/upper CI limits for the bootstrapped 1-in-X 'gev' distributions
        metric_calc='max',
        one_in_x=[10, 100],
        event_duration=(1, 'day'),
        
        ## Export
        export_method="raw",
        file_format="NetCDF",
    )
    datas.append(data)

sb_data, sc_data = datas

### Cleaning data and generating 1-in-X calculations

In [ ]:
# Cleaning the raw data from `cava_data`
clean_raw_data_sb = clean_raw_data(sb_data, sb_locs)
clean_raw_data_sc = clean_raw_data(sc_data, sc_locs)

In [ ]:
%%time
# Calculating the 1-in-X return values, and lower + upper CIs, for SB locations
all_one_in_x_sb = get_one_in_x(clean_raw_data_sb, one_in_x, event_duration, distr)
calc_data_sb = all_one_in_x_sb['return_value']

In [ ]:
%%time
# Calculating the 1-in-X return values, and lower + upper CIs, for SC locations
all_one_in_x_sc = get_one_in_x(clean_raw_data_sc, one_in_x, event_duration, distr)
calc_data_sc = all_one_in_x_sc['return_value']

In [ ]:
# Plotting the 1-in-X values by simulation for Santa Barbara locations
plot_retvals(calc_data_sb)

In [ ]:
# Plotting the 1-in-X values by simulation for Santa Clarita locations
plot_retvals(calc_data_sc)

In [ ]:
# Finding the median of the above plots along simulation dimension
plot_med_val_by_locs(calc_data_sb)

In [ ]:
plot_med_val_by_locs(calc_data_sc)

In [ ]:
### Finding the median `hour_of_year` that this 1-in-X event occurs for each simulation
med_hrs_sb = find_med_hrs(clean_raw_data_sb, all_one_in_x_sb)
med_hrs_sc = find_med_hrs(clean_raw_data_sc, all_one_in_x_sc)

In [ ]:
# Plotting the median `hour_of_year` by simulation for Santa Barbara locations
plot_retvals(med_hrs_sb, time_axis=True)

In [ ]:
# Plotting the median `hour_of_year` by simulation for Santa Clairta locations
plot_retvals(med_hrs_sc, time_axis=True)

In [ ]:
# Aggregating along simulation dimension
plot_med_val_by_locs(med_hrs_sb, time_axis=True)

In [ ]:
# Aggregating along simulation dimension
plot_med_val_by_locs(med_hrs_sc, time_axis=True)

### Inserting 1-in-X events into 8760s

In [ ]:
# Calculating the modified 8760s with the 1-in-X events inserted in
median8760_sb = clean_raw_data_sb.groupby('hour_of_year').median().median(dim='simulation').squeeze()

modified8760_sb = insert_at_hrs(
    median8760_sb,    # (location, hour_of_year)
    med_hrs_sb.median(dim='simulation'),    # (location, one_in_x)
    calc_data_sb.median(dim='simulation'),  # (location, one_in_x)
    window=12
)

In [ ]:
# Doing the same for Santa Clarita locations
median8760_sc = clean_raw_data_sc.groupby('hour_of_year').median().median(dim='simulation').squeeze()

modified8760_sc = insert_at_hrs(
    median8760_sc,    # (location, hour_of_year)
    med_hrs_sc.median(dim='simulation'),    # (location, one_in_x)
    calc_data_sc.median(dim='simulation'),  # (location, one_in_x)
    window=12
)

In [ ]:
# Plot the modified 8760s
plot_modified8760s(modified8760_sb)

In [ ]:
# Plot the modified 8760s
plot_modified8760s(modified8760_sc)